In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from itertools import combinations
import statsmodels.api as sm

In [2]:
def import_data(experimental_file, response_file, response):
    'imports data'
    'experimental_file = experimental design csv filename i.e experimental.csv'
    'response_file = results csv filename i.e Response.csv'
    'response = reponse name: i.e rheomix final deg time min or rheomix stability time min'
    
    experimental_df = pd.read_csv(experimental_file)
    response_df = pd.read_csv(response_file)
    
    X = experimental_df[experimental_df.columns.values.tolist()[1:]].values
    y = response_df[response].values
    max1 = max(y)
    min1 = min(y)

    y_norm = [2*((i-min1)/(max1-min1)) - 1 for i in y]
    
    
    X_linear = X
    
    return X, y_norm, X_linear, experimental_df, response_df
    

In [3]:
def linear_fit(y, X_linear):
    'fits model of all linear terms to obtain benchmark AIC'
    'AIC_prev_min = AIC value of previous step in stepwise regression'
    'AIC_cur_min = AIC value of current step in stepwise regression'
    
    model = sm.OLS(y, X_linear)
    results = model.fit()
    AIC_prev_min = results.aic + 1
    AIC_cur_min = results.aic
    
    return AIC_prev_min, AIC_cur_min

In [4]:
def model_type_func(array_, indexs, model_type):
    
    if len(indexs) == 2:
        
        if indexs[1]== 'inv' or indexs[1]== 'log':
            
            if indexs[1] == 'inv':
                return 1/(array_[:, indexs[0]])
            if indexs[1] == 'log':
                return np.log(array_[:, indexs[0]])
            
        else:
            if model_type == 'scheffe':
                return array_[:, indexs[0]]*array_[:, indexs[1]]
            if model_type == 'becker':
                return (array_[:, indexs[0]]*array_[:, indexs[1]])/(array_[:, indexs[0]]+array_[:, indexs[1]])

    if len(indexs) == 3:
        if model_type == 'scheffe':
            return array_[:, indexs[0]]*array_[:, indexs[1]]*array_[:, indexs[2]]
        if model_type == 'becker':
            return (array_[:, indexs[0]]*array_[:, indexs[1]]*array_[:, indexs[2]])/(array_[:, indexs[0]]+array_[:, indexs[1]]+array_[:, indexs[2]])
    
def model_terms_name(list_, terms, indexs, model_type):
    
    if len(terms) == 1:
        
        if indexs[1] == 'inv':
            list_.append(['1' + '/' + terms[0],  indexs[0], indexs[1]])
        if indexs[1] == 'log':
            list_.append(['log' + '(' + terms[0] + ')',  indexs[0], indexs[1]])

    
    if len(terms) == 2:

        if model_type == 'scheffe':
            list_.append([terms[0] + '*' + terms[1],  indexs[0], indexs[1]])
        if model_type == 'becker':
            list_.append(['(' + terms[0] + '*' + terms[1]+ ')' + '/' +  '(' + terms[0] + '+' + terms[1] + ')', indexs[0], indexs[1]])
     
    if len(terms) == 3:
        
        if model_type == 'scheffe':
            list_.append([terms[0] + '*' + terms[1] + '*' + terms[2],  indexs[0], indexs[1], indexs[2]])
        if model_type == 'becker':
            list_.append(['(' + terms[0] + '*' + terms[1] + '*' + terms[2] + ')' + '/' +  '(' + terms[0] + '+' + terms[1] + '+' + terms[2]  + ')', indexs[0], indexs[1], indexs[2]])

In [5]:
def model_terms_list(experimental_df, response_df, model_type, order, inv_log):
    'creates list of terms with key in current model'
    'creates list of possible terms with key to be added'
    
    linear_terms = experimental_df.columns.values.tolist()[1:]
    
    model_terms = []
    for i in range(len(linear_terms)):

        term = linear_terms[i]
        key = i
        model_terms.append([term, i])
     
    poss_terms = []
    for i in range(len(linear_terms)):
        for j in range(len(linear_terms)): 
            if i < j:
                
                model_terms_name(poss_terms, [linear_terms[i], linear_terms[j]], [i, j], model_type)
                
            
    if order == 3:
        for i in range(len(linear_terms)):
            for j in range(len(linear_terms)): 
                for k in range(len(linear_terms)):
                    if i < j:
                        if j < k:
                            
                            model_terms_name(poss_terms, [linear_terms[i], linear_terms[j], linear_terms[k]], [i, j, k], model_type)
                            
    
    if inv_log == 'log' or inv_log == 'inv':

        for i in range(len(linear_terms)):
            model_terms_name(poss_terms, [linear_terms[i]], [i, inv_log], model_type)

        
    return model_terms, poss_terms

In [6]:
def model_fit(experimental_file, response_file, response, model_type, order, inv_log):

    X, y, X_linear, experimental_df, response_df = import_data(experimental_file, response_file, response)
    AIC_prev_min, AIC_cur_min = linear_fit(y, X_linear)
    model_terms, poss_terms = model_terms_list(experimental_df, response_df, model_type, order, inv_log)
    

    cntt = 0
    while AIC_cur_min < AIC_prev_min:
        cntt += 1

        AIC_prev_min = AIC_cur_min

        cnt1 = 0

        for i in poss_terms:

            if len(i) == 3:

                j, k = i[1], i[2]
                add_term_cur = model_type_func(X_linear, [j, k], model_type)

                
            if len(i) == 4:

                j, k, l  = i[1], i[2], i[3]
                add_term_cur = model_type_func(X_linear, [j, k, l], model_type)

            if len(i) == 2:

                j = i[1]
                add_term_cur = X_linear[:, j]    

            X_new = np.column_stack((X, add_term_cur))
            new_model = sm.OLS(y, X_new)
            new_results = new_model.fit()
            AIC = new_results.aic
            
            
            if AIC < AIC_cur_min:

                AIC_cur_min = AIC
                X_updated = X_new
                term_key = i 
                results = new_results
                cnt1 = 1

        if AIC_cur_min < AIC_prev_min and cnt1 == 1:

            model_terms.append(term_key)
            X = X_updated
            poss_terms.remove(term_key)
            final = results

            if len(term_key) == 2:
                if inv_log == 'log' or inv_log == 'inv':
                    model_terms_name(poss_terms, [term_key[0]], [term_key[1], inv_log], model_type)
                
                for i in model_terms:
                    if len(i) ==2:
                        if i != term_key:
                            model_terms_name(poss_terms, [term_key[0], i[0]], [term_key[1], i[1]], model_type)
                        
                        
                if order == 3:
                    for i in model_terms:
                        if len(i) ==2:
                            for j in model_terms:
                                if len(j) ==2:
                                    if j != term_key and i != term_key:
                                        if model_terms.index(i) < model_terms.index(j):
                                            model_terms_name(poss_terms, [term_key[0], i[0], j[0]], [term_key[1], i[1], j[1]], model_type)


        cnt2 = 0
        cnt3 = 0
        cnt4 = 0
        for i, j in enumerate(model_terms):


            if len(j) > 2:

                X_new = np.delete(X, i, axis = 1)
                cnt3 = 2

            if len(j) == 2:

                X_new1 = np.delete(X, i, axis = 1)
                key = j[1]

                terms_key1 = [a for a in model_terms]
                terms_key1.remove(j)

                for k in model_terms:

                    if len(k) ==3:

                        if k[1] == key or k[2] == key:

                            index = terms_key1.index(k)
                            X_new2 = np.delete(X_new1, index, axis = 1)
                            X_new1 = X_new2
                            terms_key1.remove(k)
                            
                    if len(k) ==4:

                        if k[1] == key or k[2] == key or k[2] == key :

                            index = terms_key1.index(k)
                            X_new2 = np.delete(X_new1, index, axis = 1)
                            X_new1 = X_new2
                            terms_key1.remove(k)


                X_new = X_new1
                cnt3 = 1

            new_model = sm.OLS(y, X_new)
            new_results = new_model.fit()
            AIC = new_results.aic

            if AIC < AIC_cur_min:

                AIC_cur_min = AIC
                X_updated = X_new
                term_key = j

                if len(j) == 2:

                    model_key = terms_key1


                sol_results = new_results
                cnt2 = 1
                cnt4 = cnt3

        if AIC_cur_min < AIC_prev_min and cnt2 == 1:

            if cnt4 == 2:

                model_terms.remove(term_key)

                X = X_updated
                poss_terms.append(term_key)
                final = sol_results

            if cnt4 == 1:

                model_terms = model_key
                X = X_updated
                poss_terms.append(term_key)
                final = sol_results


                poss_terms_key = [a for a in poss_terms]

                for i in poss_terms_key:

                    if len(i) ==3:

                        if i[1] == term_key[1] or i[2] == term_key[1]:
                            poss_terms.remove(i)

                            
                    if len(i) ==4:
                            
                        if i[1] == term_key[1] or i[2] == term_key[1] or i[3] == term_key[1]:
                            poss_terms.remove(i)

    return final, model_terms, X, y, poss_terms

In [7]:
'model_fit(experimental_file, response_file, response, model_type, order, inv_log)'

'repsonse = rheomix final deg time min or rheomix stability time min'
test1 = 'rheomix final deg time min'
test2 = 'rheomix stability time min'

'model type = scheffe or becker'
'order = 2 or 3 for second order or special cubic'
'inv_log = inv, log or None to add inverse terms, log terms or neither'

final, terms, X, y, poss = model_fit('experimental.csv', 'Response.csv', test2, 'scheffe', 2, None)
terms, final.summary(), final.params, poss

([['Xpvc', 0],
  ['Xfiller', 1],
  ['Xstabiliser', 3],
  ['Xdinp', 4],
  ['Xldh', 5],
  ['Xdinp*Xldh', 4, 5],
  ['Xstabiliser*Xldh', 3, 5],
  ['Xfiller*Xldh', 1, 5]],
 <class 'statsmodels.iolib.summary.Summary'>
 """
                                  OLS Regression Results                                
 Dep. Variable:                      y   R-squared (uncentered):                   0.888
 Model:                            OLS   Adj. R-squared (uncentered):              0.864
 Method:                 Least Squares   F-statistic:                              37.62
 Date:                Sat, 16 Nov 2019   Prob (F-statistic):                    9.63e-16
 Time:                        10:05:17   Log-Likelihood:                          1.2433
 No. Observations:                  46   AIC:                                      13.51
 Df Residuals:                      38   BIC:                                      28.14
 Df Model:                           8                                 

In [8]:
pd.DataFrame([terms])

,0,1,2,3,4,5,6,7
0,"[Xpvc, 0]","[Xfiller, 1]","[Xstabiliser, 3]","[Xdinp, 4]","[Xldh, 5]","[Xdinp*Xldh, 4, 5]","[Xstabiliser*Xldh, 3, 5]","[Xfiller*Xldh, 1, 5]"


In [9]:
terms, final.summary(), final.params, poss

([['Xpvc', 0],
  ['Xfiller', 1],
  ['Xstabiliser', 3],
  ['Xdinp', 4],
  ['Xldh', 5],
  ['Xdinp*Xldh', 4, 5],
  ['Xstabiliser*Xldh', 3, 5],
  ['Xfiller*Xldh', 1, 5]],
 <class 'statsmodels.iolib.summary.Summary'>
 """
                                  OLS Regression Results                                
 Dep. Variable:                      y   R-squared (uncentered):                   0.888
 Model:                            OLS   Adj. R-squared (uncentered):              0.864
 Method:                 Least Squares   F-statistic:                              37.62
 Date:                Sat, 16 Nov 2019   Prob (F-statistic):                    9.63e-16
 Time:                        10:05:17   Log-Likelihood:                          1.2433
 No. Observations:                  46   AIC:                                      13.51
 Df Residuals:                      38   BIC:                                      28.14
 Df Model:                           8                                 

In [10]:
final.condition_number

1574.3648438925056

In [11]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

variables = final.model.exog
vif = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
vif 

[53.778647476556394,
 6.770462454378806,
 17.676635433136106,
 23.267532445076757,
 75.71278129778995,
 33.26466609394756,
 18.42850747970178,
 8.298723654877236]